# 基于Embedding生成歌词

基于词粒度对RNN进行训练，文本采用中文歌词的分词文本。模型中加入Embedding层来降低输入词的维度。

# 1 读取数据

In [1]:
import os, glob
import pandas as pd
from utils import *

# path  = os.getcwd() 
# path = os.path.join(path, "Chinese_Lyrics")
# stefanie_path = os.path.join(path, r"孙燕姿_9272")

# stefanie_txt = 'Stefanie.txt'
# merge_txt(stefanie_path, stefanie_txt)

path  = os.getcwd() 
path = os.path.join(path, "data")
train_txt = 'train.txt'
merge_txt(path, train_txt)

In [2]:
# train_txt = 'train.txt'
train_split = 'train_split.txt'

split_txt(train_txt, train_split)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.838 seconds.
Prefix dict has been built succesfully.


done.


In [3]:
# stefanie_data = pd.read_csv(stefanie_split, sep=' ' , header=None)

In [4]:
def load_data(fname):
    with open(fname, 'r') as f:
        text = f.read()
    
    data = text.split()
    return data

In [5]:
text = load_data(train_split)

In [6]:
print("前30个词: {}".format(text[:30]))

前30个词: ['haoxuan', '灰色', '的', '气压', ';', '扭曲', '的', '脸', ';', '好闷', '生活', '颜色', '比', '天黑', '不想', '负责', '的', '就', '无解', ';', '爱情', '像', '潜水', '氧气', '剩', '一些', ';', '无法', '呼吸', '的']


# 2 数据预处理

In [7]:
# 构造词典及映射
vocab = set(text)
vocab_to_int = {w: idx for idx, w in enumerate(vocab)}
int_to_vocab = {idx: w for idx, w in enumerate(vocab)}

In [8]:
print('Total words: {}'.format(len(text)))
print('Vocab size: {}'.format(len(vocab)))

Total words: 371408
Vocab size: 23940


In [9]:
# 转换文本为整数
int_text = [vocab_to_int[w] for w in text]

# 3 构建网络

In [10]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
import numpy as np

In [11]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow版本: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('未发现GPU，请使用GPU进行训练！')
else:
    print('默认GPU设备: {}'.format(tf.test.gpu_device_name()))

TensorFlow版本: 1.0.0
默认GPU设备: /gpu:0


## 输入层

In [12]:
def get_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, None, name='learning_rate')
    return inputs, targets, learning_rate

## RNN Cell

In [13]:
# def get_init_cell(batch_size, rnn_size):
#     '''
#     构建堆叠RNN单元
    
#     参数
#     ---
#     batch_size: 每个batch的大小
#     rnn_size: RNN隐层神经元个数
#     '''
#     lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
#     cell = tf.contrib.rnn.MultiRNNCell([lstm])
    
#     initial_state = cell.zero_state(batch_size, tf.float32)
#     initial_state = tf.identity(initial_state, 'initial_state')
#     return cell, initial_state

In [14]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    
    n_layers = 2
    def make_lstm(rnn_size):
        return tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    cell = tf.contrib.rnn.MultiRNNCell([make_lstm(rnn_size) for _ in range(n_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    return cell, initial_state


# Word Embedding

In [15]:
def get_embed(input_data, vocab_size, embed_dim):
    '''
    单词太多，需要进行embedding
    
    参数
    ---
    input_data: 输入的tensor
    vocab_size: 词汇表大小
    embed_dim: 嵌入维度
    '''
    embedding = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed

# Build RNN

In [16]:
def build_rnn(cell, inputs):
    '''
    构建RNN模型
    
    参数:
    ---
    cell: RNN单元
    inputs: 输入的batch
    '''
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, 'final_state')
    return outputs, final_state

# Build Neural Network

In [17]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    '''
    构建神经网络，将RNN层与全连接层相连
    
    参数:
    ---
    cell: RNN单元
    rnn_size: RNN隐层结点数量
    input_data: input tensor
    vocab_size
    embed_dim: 嵌入层大小
    
    '''
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    return logits, final_state

# 构造batch

在这里，我们将采用以下方式进行batch的构造，如果我们有一个1-20的序列，传入参数batch_size=3, seq_length=2的话，希望返回以下一个四维的向量。

分为了三个batch，每个batch中包含了输入和对应的目标输出。
get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)

  ## First Batch
  [
    ## Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    ## Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  ## Second Batch
  [
    ## Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    ## Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  ## Third Batch
  [
    ## Batch of Input
    [[ 5  6], [11 12], [17 18]]
    ## Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]

In [18]:
def get_batches(int_text, batch_size, seq_length):
    '''
    构造batch
    '''
    batch = batch_size * seq_length
    n_batch = len(int_text) // batch
    
    int_text = np.array(int_text[:batch * n_batch]) # 保留能构成完整batch的数量
    
    int_text_targets = np.zeros_like(int_text)
    int_text_targets[:-1], int_text_targets[-1] = int_text[1:], int_text[0]
    
    # 切分
    x = np.split(int_text.reshape(batch_size, -1), n_batch, -1)
    y = np.split(int_text_targets.reshape(batch_size, -1), n_batch, -1)
    
    return np.stack((x, y), axis=1) # 组合

In [19]:
# def get_batches1(int_text, batch_size, seq_length):
#     n_batches = int(len(int_text) / (batch_size * seq_length))
#     # Drop the last few characters to make only full batches
#     x_data = np.array(int_text[: n_batches * batch_size * seq_length])
#     y_data = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

#     x_batches = np.split(x_data.reshape(batch_size, -1), n_batches, 1)
#     y_batches = np.split(y_data.reshape(batch_size, -1), n_batches, 1)
#     return np.array(list(zip(x_batches, y_batches)))


# 3 模型训练

In [20]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 100

In [21]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab) # vocab_size
    input_text, targets, lr = get_inputs() # 输入tensor
    input_data_shape = tf.shape(input_text)
    # 初始化RNN
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # 计算softmax层概率
    probs = tf.nn.softmax(logits, name='probs')

    # 损失函数
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # 优化函数
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

In [ ]:
# 获取batch
batches = get_batches(int_text, batch_size, seq_length)
# 定义参数存储目录
save_dir = './save'

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # 每训练一定阶段对结果进行打印
            if (epoch * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch,
                    batch_i,
                    len(batches),
                    train_loss))
    # 保存模型
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/145   train_loss = 10.083
Epoch   0 Batch  100/145   train_loss = 7.360
Epoch   1 Batch   55/145   train_loss = 7.518
Epoch   2 Batch   10/145   train_loss = 7.017
Epoch   2 Batch  110/145   train_loss = 7.243
Epoch   3 Batch   65/145   train_loss = 6.701
Epoch   4 Batch   20/145   train_loss = 6.656
Epoch   4 Batch  120/145   train_loss = 6.310
Epoch   5 Batch   75/145   train_loss = 6.136
Epoch   6 Batch   30/145   train_loss = 6.221
Epoch   6 Batch  130/145   train_loss = 5.878
Epoch   7 Batch   85/145   train_loss = 5.739
Epoch   8 Batch   40/145   train_loss = 5.667
Epoch   8 Batch  140/145   train_loss = 5.595
Epoch   9 Batch   95/145   train_loss = 5.128
Epoch  10 Batch   50/145   train_loss = 4.874
Epoch  11 Batch    5/145   train_loss = 5.047
Epoch  11 Batch  105/145   train_loss = 4.633
Epoch  12 Batch   60/145   train_loss = 4.792
Epoch  13 Batch   15/145   train_loss = 4.624
Epoch  13 Batch  115/145   train_loss = 4.395
Epoch  14 Batch   70/145   train_

In [40]:
def get_tensors(loaded_graph):
    '''
    获取模型训练结果参数
    
    参数
    ---
    loaded_graph: 从文件加载的tensroflow graph
    '''
    inputs = loaded_graph.get_tensor_by_name('inputs:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
    return inputs, initial_state, final_state, probs

In [42]:
def pick_word(probabilities, int_to_vocab):
    '''
    选择单词进行文本生成，用来以一定的概率生成下一个词
    
    参数
    ---
    probabilities: Probabilites of the next word
    int_to_vocab: 映射表
    '''
    result = np.random.choice(len(probabilities), 50, p=probabilities)
    return int_to_vocab[result[0]]

In [51]:
# 生成文本的长度
gen_length = 300

# 定义冷启动的单词
prime_word = '爱情'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # 加载模型
    loader = tf.train.import_meta_graph(save_dir + '.meta')
    loader.restore(sess, save_dir)

    # 获取训练的结果参数
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word]
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # 生成句子
    for n in range(gen_length):
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # 预测
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    lyrics = ' '.join(gen_sentences)
    lyrics = lyrics.replace(';', '\n')
    lyrics = lyrics.replace('.', ' ')
    # lyrics = lyrics.replace(' ', '')
    print(lyrics)
    

爱情 到底 让 人 脆弱 还是 让 人 坚定 我 用尽 最后 的 力气 从头到尾 都 没有 哭泣 爱 曾 是 怕 多 孙燕姿 了 又 突然 失重 在 飘 我 说 我 让 快乐 一起 有 ~ 星期一 天气 晴 我 离开 你 不带 任何 行李 除了 一本 陪 我 放逐 的 日记 伤心 让 人 不想 爱 自己 那么 也 只好 暂时 不爱 你 拉开距离 等 着 有 一天 忽然 想起 你 离开 的 原因 再也 想不起 再 翻出 旧 的 日记 从 新 写 起 星期一 天气 晴 我 离开 你 突然 就 下 了 决心 我 在 日历 上面 画 下 星星 星期一 天气 晴 我 离开 你 不带 任何 行李 除了 一本 陪 我 放逐 的 日记 伤心 让 人 不想 爱 自己 那么 也 只好 暂时 不爱 你 拉开距离 等 着 有 一天 忽然 想起 你 离开 的 原因 再也 想不起 再 翻出 旧 的 日记 从 新 写 起 星期一 天气 晴 我 离开 你 突然 就 下 了 决心 我 在 日历 上面 画 下 星星 星期一 天气 晴 我 离开 你 不带 任何 行李 除了 一本 陪 我 放逐 的 日记 伤心 让 人 不想 爱 自己 那么 也 只好 暂时 不爱 你 拉开距离 等 着 有 一天 忽然 想起 你 离开 的 原因 再也 想不起 再 翻出 旧 的 日记 从 新 写 起 星期一 天气 晴 我 离开 你 突然 就 下 了 决心 我 在 日历 上面 画 下 星星 星期一 天气 晴 我 离开 你 不带 任何 行李 除了 一本 陪 我 放逐 的 日记 伤心 让 人 不想 爱 自己 那么 也 只好 暂时 不爱 你 拉开距离 等 着 有 一天 忽然 想起 你 离开 的 原因 再也 想不起 再 翻出 旧 的 日记 从 新 写 起 星期一 天气 晴 我 离开 你 突然 就 下
